In [54]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Xavier initialization
def xavier_init(tensor):
    if isinstance(tensor, nn.Linear):
        nn.init.xavier_normal_(tensor.weight)
        if tensor.bias is not None:
            tensor.bias.data.fill_(0.01)


In [55]:
class Encoder(nn.Module):
    def __init__(self, image_dim, label_dim, hidden_dim, latent_dim):
        super(Encoder, self).__init__()
        self.fc1 = nn.Linear(image_dim + label_dim, hidden_dim)
        self.fc2_mean = nn.Linear(hidden_dim, latent_dim)
        self.fc2_logvar = nn.Linear(hidden_dim, latent_dim)

        self.apply(xavier_init)

    def forward(self, x, y):
        x = torch.cat([x, y], dim=1)
        h = F.relu(self.fc1(x))
        mean = self.fc2_mean(h)
        logvar = self.fc2_logvar(h)
        return mean, logvar


In [56]:
class Decoder(nn.Module):
    def __init__(self, latent_dim, label_dim, hidden_dim, image_dim):
        super(Decoder, self).__init__()
        self.fc1 = nn.Linear(latent_dim + label_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, image_dim)

        self.apply(xavier_init)

    def forward(self, z, y):
        z = torch.cat([z, y], dim=1)
        h = F.relu(self.fc1(z))
        x_recon = torch.sigmoid(self.fc2(h))
        return x_recon


In [57]:
class CVAE(nn.Module):
    def __init__(self, image_dim, label_dim, hidden_dim, latent_dim):
        super(CVAE, self).__init__()
        self.encoder = Encoder(image_dim, label_dim, hidden_dim, latent_dim)
        self.decoder = Decoder(latent_dim, label_dim, hidden_dim, image_dim)
        self.latent_dim = latent_dim

    def reparameterize(self, mean, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mean + eps * std

    def forward(self, x, y):
        mean, logvar = self.encoder(x, y)
        z = self.reparameterize(mean, logvar)
        x_recon = self.decoder(z, y)
        return x_recon, mean, logvar


In [58]:
def loss_function(x_recon, x, mean, logvar):
    BCE = F.binary_cross_entropy(x_recon, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mean.pow(2) - logvar.exp())
    return BCE + KLD


In [60]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, labels) in enumerate(train_loader):
        data = data.view(-1, 784)
        labels = F.one_hot(labels, num_classes=10).float()
        optimizer.zero_grad()
        recon_batch, mean, logvar = model(data, labels)
        loss = loss_function(recon_batch, data, mean, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item() / len(data):.6f}')

    print(f'====> Epoch: {epoch} Average loss: {train_loss / len(train_loader.dataset):.4f}')


In [61]:
def main():
    # Load and preprocess the MNIST dataset
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Lambda(lambda x: x.view(-1))
    ])
    
    train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
    test_dataset = datasets.MNIST(root='./data', train=False, transform=transform)
    
    train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

    # Define dimensions
    image_dim = 784  # 28x28 images flattened
    label_dim = 10  # One-hot encoded labels for 10 classes
    hidden_dim = 512  # Hidden layer size
    latent_dim = 2  # Latent space size

    # Create the CVAE model
    model = CVAE(image_dim, label_dim, hidden_dim, latent_dim).cuda()

    # Define optimizer
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    # Training the model
    epochs = 10
    for epoch in range(1, epochs + 1):
        train(model, train_loader, optimizer, epoch)

if __name__ == "__main__":
    main()


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 25942500.82it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 666767.73it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:02<00:00, 800713.78it/s] 


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 916507.69it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



AssertionError: Torch not compiled with CUDA enabled